In [1]:

# Set up notebook
%pprint
import sys
if ('../py' not in sys.path): sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

# load libraries
from FRVRS import nu, fu
from numpy import nan
from pandas import DataFrame, read_csv, read_excel, concat, get_dummies, isna
from re import split, search, sub
from scipy.stats import f_oneway, ttest_ind, kruskal, norm
import itertools
import os.path as osp
import statsmodels.api as sm

In [3]:

# load data frames
data_frames_dict = nu.load_data_frames(
    verbose=True, metrics_evaluation_open_world_csv_stats_df=''
)
csv_stats_df = data_frames_dict['metrics_evaluation_open_world_csv_stats_df'].copy()
print(sorted([cn for cn in csv_stats_df.columns if 'all' in cn]))

No pickle exists for metrics_evaluation_open_world_csv_stats_df - attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/metrics_evaluation_open_world_csv_stats_df.csv.
[]


In [4]:

# load data frames
data_frames_dict = nu.load_data_frames(
    verbose=True, metrics_evaluation_open_world_scene_stats_df=''
)
scene_stats_df = data_frames_dict['metrics_evaluation_open_world_scene_stats_df'].copy()
print(sorted([cn for cn in scene_stats_df.columns if 'all' in cn]))

No pickle exists for metrics_evaluation_open_world_scene_stats_df - attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/metrics_evaluation_open_world_scene_stats_df.csv.
['all_patient_injuries_count']


In [5]:

groupby_columns = ['participant_id', 'session_uuid', 'scene_id']
rows_list = []
for (participant_id, session_uuid, scene_id), scene_df in scene_stats_df.groupby(groupby_columns):
    row_dict = {}
    for cn in groupby_columns: row_dict[cn] = eval(cn)
    for cn in scene_df.columns:
        if 'injur' in cn:
            row_dict[cn] = scene_df[cn].squeeze()
    rows_list.append(row_dict)
injury_counts_df = DataFrame(rows_list)

In [6]:

columns_list = [cn for cn in injury_counts_df.columns if 'injur' in cn]
injury_counts_df[columns_list]

,all_patient_injuries_count,injury_correctly_treated_count,injury_not_treated_count,injury_wrongly_treated_count
0,31,15,10,6
1,18,0,18,0
2,0,0,0,0
3,0,0,0,0
4,23,11,8,4
...,...,...,...,...
63,19,5,9,5
64,23,12,7,4
65,31,23,2,6
66,31,10,20,1


In [5]:

groupby_columns = ['participant_id', 'session_uuid', 'scene_id']
millisecond_threshold = 3
for (participant_id, session_uuid, scene_id), scene_df in csv_stats_df.groupby(groupby_columns):
    
    # Loop through each injury and make a determination if it's treated or not
    injury_correctly_treated_count = 0
    for patient_id, patient_df in scene_df.groupby('patient_id'):
        for injury_id, injury_df in patient_df.groupby('injury_id'):
            is_correctly_treated = fu.get_is_injury_correctly_treated(injury_df, patient_df)
            if is_correctly_treated: injury_correctly_treated_count += 1

In [6]:

mask_series = (injury_df.action_type == 'INJURY_TREATED')
action_ticks_list = sorted(injury_df[mask_series].action_tick.unique())
mask_series = ~injury_df.injury_required_procedure.isnull()
required_procedure = injury_df[mask_series].injury_required_procedure.mode().squeeze()
print(required_procedure)
for action_tick in action_ticks_list:
    mask_series = patient_df.action_tick.map(
        lambda ts: abs(ts - action_tick) < millisecond_threshold
    ) & patient_df.action_type.isin(['TOOL_APPLIED'])
    if mask_series.any():
        print([(required_procedure == self.tool_type_to_required_procedure_dict.get(tool_type)) for tool_type in patient_df[mask_series].tool_applied_type])
        is_correctly_treated = any(
            [(required_procedure == self.tool_type_to_required_procedure_dict.get(tool_type)) for tool_type in patient_df[mask_series].tool_applied_type]
        )

none


In [7]:

injury_df.dropna(axis='columns', how='all').T

,42508
action_type,INJURY_TREATED
action_tick,1286988
event_time,2024-03-14 10:57:00
session_uuid,23081f6e-875e-44f5-8bd0-edc3905f5c2c
csv_file_subpath,Human_Sim_Metrics_Data_4-12-2024/23081f6e-875e...
injury_treated_id,Unspecified
injury_treated_patient_id,Civilian 1 Female Root
injury_treated_required_procedure,none
injury_treated_severity,medium
injury_treated_body_region,leftArm


In [22]:

mask_series = (csv_stats_df.injury_required_procedure == 'none')
csv_stats_df[mask_series].injury_id.unique()

array(['Unspecified', 'Ear Bleed'], dtype=object)

In [16]:

columns_list = ['action_type', 'action_tick']
patient_df[columns_list]

,action_type,action_tick
42381,PATIENT_ENGAGED,1262495
42407,BREATHING_CHECKED,1266435
42408,INJURY_TREATED,1267526
42409,TOOL_APPLIED,1267526
42420,INJURY_TREATED,1273515
42422,PULSE_TAKEN,1273567
42483,INJURY_TREATED,1281386
42485,PULSE_TAKEN,1281897
42508,INJURY_TREATED,1286988
42626,PATIENT_DEMOTED,192305


In [29]:

from pandas import Series

groupby_columns = ['participant_id', 'session_uuid', 'scene_id', 'patient_id']
for (participant_id, session_uuid, scene_id, patient_id), patient_df in csv_stats_df.groupby(groupby_columns):
    for injury_id, injury_df in patient_df.groupby('injury_id'):
        action_ticks_list = sorted(injury_df.action_tick.unique())
        mask_series = ~patient_df.tool_type.isnull() & patient_df.action_tick.isin(action_ticks_list)
        if mask_series.any():
            df = patient_df[mask_series]
            mask_series = ~injury_df.injury_record_required_procedure.isnull()
            required_procedure = injury_df[mask_series].injury_record_required_procedure.mode().squeeze()
            if isinstance(required_procedure, Series):
                required_procedure = '(who knows?)'
                raise
            for tool_type in df.tool_type:
                is_correctly_treated = (required_procedure == fu.tool_type_to_required_procedure_dict.get(tool_type))
                print(
                    f'{patient_id} has a {injury_id} which requires a {required_procedure} and it {"was" if is_correctly_treated else "was not"} correctly treated'
                )
                raise

Adept Shooter Root has a R Bicep Puncture which requires a tourniquet and it was correctly treated


RuntimeError: No active exception to reraise

In [30]:

columns_list = [cn for cn in injury_df.columns if 'required_procedure' in cn]
injury_df[columns_list]

,injury_record_required_procedure,injury_treated_required_procedure,injury_required_procedure
42658,tourniquet,NaN,tourniquet
44199,NaN,tourniquet,tourniquet


In [54]:

osp.basename(osp.join('../data/logs', df.csv_file_subpath.squeeze()))

'04f80090-9e61-431d-8473-dccb75fed04d_2024207.csv'